# Assignment 2 - Problem A (More Concise, Read Data from External Sources)

## Step 1: Import PuLP modeler functions

In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.5.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.0
pytest: 3.5.1
pip: 18.1
setuptools: 40.2.0
Cython: 0.28.2
numpy: 1.14.3
scipy: 1.1.0
pyarrow: 0.10.0
xarray: None
IPython: 6.4.0
sphinx: 1.7.4
patsy: 0.5.0
dateutil: 2.6.1
pytz: 2018.4
blosc: None
bottleneck: 1.2.1
tables: 3.4.3
numexpr: 2.6.5
feather: 0.4.0
matplotlib: 2.2.2
openpyxl: 2.5.3
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.4
lxml: 4.2.1
bs4: 4.6.0
html5lib: 1.0.1
sqlalchemy: 1.2.7
pymysql: 0.9.2
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


## Step 2: Define a variable using LpProblem to represent the optimization problem

- give the variable a name, for examle **probA**
- use **LpProblem** class: **LpProblem(name='NoName', sense=1)**. Parameters are explained below:
- **name**:name of the problem used in the output .lp file
- **sense**: type of the LP problem objective. Either LpMinimize (default) or LpMaximize.
- LpProblem returns an LP problem

In [6]:
probA=LpProblem("Problem A",LpMaximize)

## Step 3: Define decision variables using LpVariable
- give each decision variable a name, say **xhc** represents the number passengers to fly from Houston to Chicago
- use **LpVariable** class: **LpVariable(name, lowBound=None, upBound=None, cat='Continuous', e=None)**. Parameters are explained below:
- **name**:The name of the variable used in the output .lp file
- **lowBound**: The lower bound on this variable’s range. Default is negative infinity
- **upBound**: The upper bound on this variable’s range. Default is positive infinity
- **cat**:  The category this variable is in, Integer, Binary or Continuous(default)
- **e**: Used for column based modelling: relates to the variable’s existence in the objective function and constraints

In [7]:
#import os
#os.getcwd()
#read data from excel file that is saved in the same directory, saved as Microsoft Excel Worksheet,
# Not Microsoft Excel 97-2003 Worksheet'''
df_rev=pd.read_excel('Data_BlueSkyAirlines_Network_py.xlsx',sheet_name='revenue')  # can read xlsx saved as Microsoft Excel Worksheet, Not Microsoft Excel 97-2003 Worksheet
df_demand=pd.read_excel('Data_BlueSkyAirlines_Network_py.xlsx',sheet_name='demand') 
#
print(df_rev)
print(df_demand)
# print the dataframe to help figure out iloc value
Fares = [np.array(df_rev.iloc[4+i,3:7],dtype=float) for i in range(4)]
MaxDemand =[np.array(df_demand.iloc[5+i,3:7],dtype=float) for i in range(4)] # Depending on the version, 4+i need to be changed to 5+i 
print(Fares)
print(MaxDemand)
#Demand =np.array(df.iloc[1,1:7],dtype=float)

   BlueSky Airlines: Network Revenue Management (A)  \
0                                               NaN   
1                                               NaN   
2                                               NaN   
3                                               NaN   
4                                               NaN   
5                                               NaN   
6                                               NaN   
7                                               NaN   

                   Unnamed: 1 Unnamed: 2   Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0                         NaN        NaN          NaN        NaN        NaN   
1  revenue for each city pair        NaN          NaN        NaN        NaN   
2                         NaN        NaN  Destination        NaN        NaN   
3                         NaN        NaN      Houston    Chicago      Miami   
4                      Origin    Houston            0        197        110   
5                         NaN 

In [8]:
# creates a list of all cities
City=['H','C','M','P']
# Creates a list of tuples containing all the possible routes for transport
Routes = [(fr, to) for fr in City for to in City]

MaxDemand= makeDict([City,City],MaxDemand)

Fares = makeDict([City,City],Fares,0)
Capacity = 240
passenger_vars = LpVariable.dicts("x",(City,City),lowBound=0,cat='Continuous')

In [9]:
#objective function
probA+=lpSum([passenger_vars[fr][to]*Fares[fr][to] for (fr,to) in Routes])
# outbound capacity constraint
for i in City[1:]:
    probA += lpSum([passenger_vars[i][j] for j in City]) <= Capacity,"outbound_%s"%i

# inbound capacity constraint    
for j in City[1:]:
    probA += lpSum([passenger_vars[i][j] for i in City]) <= Capacity,"inbound_%s"%j

# demand constraint
for i in City:
    for j in City:
        probA += passenger_vars[i][j] <= MaxDemand[i][j]

## Step 5: Run solver

- use name.solve(solver=None), where name is the LP problem variable defined by LpProblem
- Solve the given Lp problem. 
- This function changes the problem to make it suitable for solving then calls the solver.actualSolve method to find the solution. 
- solver – Optional: the specific solver to be used, defaults to the default solver.

In [10]:
probA.writeLP("Bluesky3.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


## Step 6: Print the optiomal solution

In [11]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total revenue=", value(probA.objective))

x_C_C = 0.0 	Reduced Cost = -382.0
x_C_H = 84.0 	Reduced Cost = 0.0
x_C_M = 94.0 	Reduced Cost = 0.0
x_C_P = 62.0 	Reduced Cost = 0.0
x_H_C = 123.0 	Reduced Cost = 0.0
x_H_H = 0.0 	Reduced Cost = 0.0
x_H_M = 80.0 	Reduced Cost = 0.0
x_H_P = 110.0 	Reduced Cost = 0.0
x_M_C = 100.0 	Reduced Cost = 0.0
x_M_H = 72.0 	Reduced Cost = 0.0
x_M_M = 0.0 	Reduced Cost = -192.0
x_M_P = 68.0 	Reduced Cost = 0.0
x_P_C = 17.0 	Reduced Cost = 0.0
x_P_H = 115.0 	Reduced Cost = 0.0
x_P_M = 66.0 	Reduced Cost = 0.0
x_P_P = 0.0 	Reduced Cost = -5.0
Total revenue= 185593.0


In [12]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Sensitivity Analysis
Constraint		Shadow Price	Slack
outbound_C : x_C_C + x_C_H + x_C_M + x_C_P <= 240 	 190.0 		 -0.0
outbound_M : x_M_C + x_M_H + x_M_M + x_M_P <= 240 	 100.0 		 -0.0
outbound_P : x_P_C + x_P_H + x_P_M + x_P_P <= 240 	 -0.0 		 42.0
inbound_C : x_C_C + x_H_C + x_M_C + x_P_C <= 240 	 192.0 		 -0.0
inbound_M : x_C_M + x_H_M + x_M_M + x_P_M <= 240 	 92.0 		 -0.0
inbound_P : x_C_P + x_H_P + x_M_P + x_P_P <= 240 	 5.0 		 -0.0
_C1 : x_H_H <= -0.0 	 -0.0 		 -0.0
_C2 : x_H_C <= 123.0 	 5.0 		 -0.0
_C3 : x_H_M <= 80.0 	 18.0 		 -0.0
_C4 : x_H_P <= 110.0 	 120.0 		 -0.0
_C5 : x_C_H <= 130.0 	 -0.0 		 46.0
_C6 : x_C_C <= -0.0 	 -0.0 		 -0.0
_C7 : x_C_M <= 98.0 	 -0.0 		 4.0
_C8 : x_C_P <= 88.0 	 -0.0 		 26.0
_C9 : x_M_H <= 72.0 	 8.0 		 -0.0
_C10 : x_M_C <= 105.0 	 -0.0 		 5.0
_C11 : x_M_M <= -0.0 	 -0.0 		 -0.0
_C12 : x_M_P <= 68.0 	 133.0 		 -0.0
_C13 : x_P_H <= 115.0 	 110.0 		 -0.0
_C14 : x_P_C <= 90.0 	 -0.0 		 73.0
_C15 : x_P_M <= 66.0 	 138.0 		 -0.0
_C16 : x_P_P <= -0.0 	

## Problem C


$$ \sum_{i=H,C,M,P,j=H,C,M,P,} r_{ij}x_{ij} -\sum_{i=C,M,P}(24000+80y_i)\\
s.t. \\
Capacity(HC): x_{HC}+x_{MC}+x_{PC} -y_C \le 240 \\
Capacity(HM): x_{HM}+x_{CM}+x_{PM} -y_M \le 240 \\
Capacity(HP): x_{HP}+x_{CP}+x_{MP} -y_P \le 240 \\
Capacity(CH): x_{CH}+x_{CM}+x_{CP} -y_C\le 240 \\
Capacity(MH): x_{MH}+x_{MC}+x_{MP} -y_H\le 240 \\
Capacity(PH): x_{PH}+x_{PC}+x_{PM} -y_P\le 240 \\
Demand: x_{ij} \le d_{ij}\\
$$

In [40]:
probC=LpProblem("Problem C",LpMaximize)
y_vars=LpVariable.dict("y_%s",["C","M","P"],lowBound=0)
probC+=lpSum([passenger_vars[fr][to]*Fares[fr][to] for (fr,to) in Routes])-lpSum([24000+80*y_vars[i] for i in ["C","M","P"]]), "profit"
for i in City[1:]:
    probC += lpSum([passenger_vars[i][j] for j in City])-y_vars[i] <= 0,"outbound_%s"%i

# inbound capacity constraint    
for j in City[1:]:
    probC += lpSum([passenger_vars[i][j] for i in City])-y_vars[j] <= 0,"inbound_%s"%j

# demand constraint
for i in City:
    for j in City:
        probC += passenger_vars[i][j] <= MaxDemand[i][j],"demand_%s to %s"%(i,j)
statusC=probC.solve()
probC.writeLP("Bluesky3C.lp")
print("Status:",LpStatus[probC.status])

Status: Optimal


In [41]:
for v in probC.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total revenue=", value(probC.objective))

x_C_C = 0.0 	Reduced Cost = -80.0
x_C_H = 130.0 	Reduced Cost = 0.0
x_C_M = 98.0 	Reduced Cost = 0.0
x_C_P = 88.0 	Reduced Cost = 0.0
x_H_C = 123.0 	Reduced Cost = 0.0
x_H_H = 0.0 	Reduced Cost = 0.0
x_H_M = 80.0 	Reduced Cost = 0.0
x_H_P = 110.0 	Reduced Cost = 0.0
x_M_C = 105.0 	Reduced Cost = 0.0
x_M_H = 72.0 	Reduced Cost = 0.0
x_M_M = 0.0 	Reduced Cost = -80.0
x_M_P = 68.0 	Reduced Cost = 0.0
x_P_C = 90.0 	Reduced Cost = 0.0
x_P_H = 115.0 	Reduced Cost = 0.0
x_P_M = 66.0 	Reduced Cost = 0.0
x_P_P = 0.0 	Reduced Cost = -80.0
y_C = 318.0 	Reduced Cost = 0.0
y_M = 245.0 	Reduced Cost = 0.0
y_P = 271.0 	Reduced Cost = 0.0
Total revenue= 77287.0


In [42]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Sensitivity Analysis
Constraint		Shadow Price	Slack
outbound_C : x_C_C + x_C_H + x_C_M + x_C_P <= 240 	 190.0 		 -0.0
outbound_M : x_M_C + x_M_H + x_M_M + x_M_P <= 240 	 100.0 		 -0.0
outbound_P : x_P_C + x_P_H + x_P_M + x_P_P <= 240 	 -0.0 		 42.0
inbound_C : x_C_C + x_H_C + x_M_C + x_P_C <= 240 	 192.0 		 -0.0
inbound_M : x_C_M + x_H_M + x_M_M + x_P_M <= 240 	 92.0 		 -0.0
inbound_P : x_C_P + x_H_P + x_M_P + x_P_P <= 240 	 5.0 		 -0.0
_C1 : x_H_H <= -0.0 	 -0.0 		 -0.0
_C2 : x_H_C <= 123.0 	 5.0 		 -0.0
_C3 : x_H_M <= 80.0 	 18.0 		 -0.0
_C4 : x_H_P <= 110.0 	 120.0 		 -0.0
_C5 : x_C_H <= 130.0 	 -0.0 		 46.0
_C6 : x_C_C <= -0.0 	 -0.0 		 -0.0
_C7 : x_C_M <= 98.0 	 -0.0 		 4.0
_C8 : x_C_P <= 88.0 	 -0.0 		 26.0
_C9 : x_M_H <= 72.0 	 8.0 		 -0.0
_C10 : x_M_C <= 105.0 	 -0.0 		 5.0
_C11 : x_M_M <= -0.0 	 -0.0 		 -0.0
_C12 : x_M_P <= 68.0 	 133.0 		 -0.0
_C13 : x_P_H <= 115.0 	 110.0 		 -0.0
_C14 : x_P_C <= 90.0 	 -0.0 		 73.0
_C15 : x_P_M <= 66.0 	 138.0 		 -0.0
_C16 : x_P_P <= -0.0 	